In [6]:
!pip install rdkit
!pip install /kaggle/input/rdkit-install-whl/rdkit_wheel/rdkit_pypi-2022.9.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl


[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
ERROR: rdkit_pypi-2022.9.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl is not a supported wheel on this platform.

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [7]:
!pip install lightgbm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 25.1 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [8]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import Descriptors, rdMolDescriptors
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
import lightgbm as lgb
import warnings
warnings.filterwarnings('ignore')

In [9]:
molecules = [
    ('CCO', 'Ethanol - simple alcohol'),
    ('CCCCCCCC', 'Octane - long chain'),
    ('c1ccccc1', 'Benzene - aromatic ring'),
]

for smiles, description in molecules:
    mol = Chem.MolFromSmiles(smiles)
    
    print(f"\n{description}")
    print(f"SMILES: {smiles}")
    print(f"  Molecular Weight: {Descriptors.MolWt(mol):.1f}")
    print(f"  LogP (oiliness): {Descriptors.MolLogP(mol):.2f}")
    print(f"  Rotatable Bonds: {Descriptors.NumRotatableBonds(mol)}")
    print(f"  Aromatic Rings: {Descriptors.NumAromaticRings(mol)}")
    print(f"  Complexity (BertzCT): {Descriptors.BertzCT(mol):.0f}")


Ethanol - simple alcohol
SMILES: CCO
  Molecular Weight: 46.1
  LogP (oiliness): -0.00
  Rotatable Bonds: 0
  Aromatic Rings: 0
  Complexity (BertzCT): 3

Octane - long chain
SMILES: CCCCCCCC
  Molecular Weight: 114.2
  LogP (oiliness): 3.37
  Rotatable Bonds: 5
  Aromatic Rings: 0
  Complexity (BertzCT): 25

Benzene - aromatic ring
SMILES: c1ccccc1
  Molecular Weight: 78.1
  LogP (oiliness): 1.69
  Rotatable Bonds: 0
  Aromatic Rings: 1
  Complexity (BertzCT): 72


In [10]:
# Load data
train_df = pd.read_csv('/kaggle/input/neurips-open-polymer-prediction-2025/train.csv')
test_df = pd.read_csv('/kaggle/input/neurips-open-polymer-prediction-2025/test.csv')

In [11]:
from rdkit.Chem import Descriptors
from rdkit import Chem
import numpy as np

def get_molecular_descriptors(max_autocorr=10):
    """Get molecular descriptors - either hardcoded list or auto-discovered"""

    descriptor_list_all = []
    test_mol = Chem.MolFromSmiles('CCO')

    # Collect all valid descriptors first
    for name in dir(Descriptors):
        if not name.startswith('_'):
            try:
                func = getattr(Descriptors, name)
                if callable(func):
                    result = func(test_mol)
                    if isinstance(result, (int, float)) and not np.isnan(result):
                        descriptor_list_all.append((name, func))
            except:
                pass

    print(f"🔍 Total discovered descriptors before filtering: {len(descriptor_list_all)}")

    # Sort AUTOCORR2D descriptors by their numeric suffix
    autocorr_descriptors = [
        (name, func)
        for name, func in descriptor_list_all
        if name.startswith('AUTOCORR2D_')
    ]
    autocorr_descriptors.sort(key=lambda x: int(x[0].split('_')[-1]))

    # Select only the lowest-numbered ones
    limited_autocorr = autocorr_descriptors[:max_autocorr]

    # Include all other descriptors
    other_descriptors = [
        (name, func)
        for name, func in descriptor_list_all
        if not name.startswith('AUTOCORR2D_')
    ]

    # Final descriptor list
    descriptor_list = limited_autocorr + other_descriptors

    print(f"✅ Auto-discovered {len(descriptor_list)} descriptors (limited to {max_autocorr} AUTOCORR2D):")
    names = [name for name, _ in descriptor_list]
    print("  " + ", ".join(names))

    feature_names = [name for name, _ in descriptor_list]
    return descriptor_list, feature_names

molecular_descriptors =  get_molecular_descriptors(max_autocorr=10)

🔍 Total discovered descriptors before filtering: 409
✅ Auto-discovered 227 descriptors (limited to 10 AUTOCORR2D):
  AUTOCORR2D_1, AUTOCORR2D_2, AUTOCORR2D_3, AUTOCORR2D_4, AUTOCORR2D_5, AUTOCORR2D_6, AUTOCORR2D_7, AUTOCORR2D_8, AUTOCORR2D_9, AUTOCORR2D_10, AvgIpc, BCUT2D_CHGHI, BCUT2D_CHGLO, BCUT2D_LOGPHI, BCUT2D_LOGPLOW, BCUT2D_MRHI, BCUT2D_MRLOW, BCUT2D_MWHI, BCUT2D_MWLOW, BalabanJ, BertzCT, Chi0, Chi0n, Chi0v, Chi1, Chi1n, Chi1v, Chi2n, Chi2v, Chi3n, Chi3v, Chi4n, Chi4v, EState_VSA1, EState_VSA10, EState_VSA11, EState_VSA2, EState_VSA3, EState_VSA4, EState_VSA5, EState_VSA6, EState_VSA7, EState_VSA8, EState_VSA9, ExactMolWt, FpDensityMorgan1, FpDensityMorgan2, FpDensityMorgan3, FractionCSP3, HallKierAlpha, HeavyAtomCount, HeavyAtomMolWt, Ipc, Kappa1, Kappa2, Kappa3, LabuteASA, MaxAbsEStateIndex, MaxAbsPartialCharge, MaxEStateIndex, MaxPartialCharge, MinAbsEStateIndex, MinAbsPartialCharge, MinEStateIndex, MinPartialCharge, MolLogP, MolMR, MolWt, NHOHCount, NOCount, NumAliphaticCarbo

In [12]:
def smiles_to_features(smiles_list, descriptor_functions):
   """Convert SMILES strings to raw feature matrix"""
   
   features = []
   total = len(smiles_list)
   
   print(f"Processing {total} SMILES...", end="", flush=True)
   
   for i, smiles in enumerate(smiles_list):
       # Progress indicator every 1000 molecules or at milestones
       if i > 0 and (i % 1000 == 0 or i == total - 1):
           print(f" {i+1}/{total}", end="", flush=True)
       
       mol_features = []
       try:
           mol = Chem.MolFromSmiles(smiles)
           if mol is None:
               # Invalid SMILES - fill with NaN
               mol_features = [np.nan] * len(descriptor_functions)
           else:
               # Calculate each descriptor
               for name, func in descriptor_functions:
                   try:
                       value = func(mol)
                       # Handle problematic values
                       if np.isinf(value) or abs(value) > 1e10:
                           value = np.nan
                       mol_features.append(value)
                   except:
                       # Descriptor calculation failed
                       mol_features.append(np.nan)
       except:
           # Complete failure - fill entire row with NaN
           mol_features = [np.nan] * len(descriptor_functions)
       
       features.append(mol_features)
   
   print(" ✅", flush=True)
   return np.array(features, dtype=float)

descriptor_functions, feature_names = molecular_descriptors
X_raw = smiles_to_features(train_df['SMILES'].values, descriptor_functions)    

Processing 7973 SMILES... 1001/7973 2001/7973 3001/7973 4001/7973 5001/7973 6001/7973 7001/7973 7973/7973 ✅


In [13]:
def clean_features(X):
   """Handle NaN/inf values and impute missing data"""
   X[np.isinf(X)] = np.nan
   
   # Count and report missing values
   missing = np.isnan(X).sum()
   print(f"🧹 Cleaned {missing:,} missing values ({missing/X.size*100:.1f}%)")
   
   # Median imputation
   for i in range(X.shape[1]):
       col = X[:, i]
       if np.isnan(col).any():
           X[np.isnan(col), i] = np.nanmedian(col) if not np.isnan(np.nanmedian(col)) else 0
   
   return X

X = clean_features(X_raw)


🧹 Cleaned 96,915 missing values (5.4%)


In [14]:
def train_target_property(X, train_df, target_name):
    
    # Get targets (only non-null values)
    mask = train_df[target_name].notna()
    
    X_target = X[mask]
    y_target = train_df[target_name].values[mask]

    print(f"📊 Training on {len(y_target)} samples with {target_name} values")
    print(f"📈 Target range: {y_target.min():.4f} to {y_target.max():.4f}")
    
    # Scale features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_target)
    
    # LightGBM parameters
    params = {
        'objective': 'regression',
        'metric': 'mae',
        'boosting_type': 'gbdt',
        'num_leaves': 127,           
        'learning_rate': 0.07,       
        'feature_fraction': 0.8,     
        'bagging_fraction': 0.9,     
        'bagging_freq': 1,           # Bag every iteration
        'lambda_l1': 0.1,            # L1 regularization
        'lambda_l2': 0.1,            # L2 regularization
        'min_data_in_leaf': 10,      # Prevent overfitting
        'verbose': -1,
        'random_state': 42
    }
    
    # 5-fold cross-validation
    cv_scores = []
    models = []
    
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    for fold, (train_idx, val_idx) in enumerate(kf.split(X_scaled)):
        X_train, X_val = X_scaled[train_idx], X_scaled[val_idx]
        y_train, y_val = y_target[train_idx], y_target[val_idx]
        
        train_data = lgb.Dataset(X_train, label=y_train)
        val_data = lgb.Dataset(X_val, label=y_val, reference=train_data)
        
        model = lgb.train(
            params,
            train_data,
            valid_sets=[val_data],
            num_boost_round=2000,
            callbacks=[lgb.early_stopping(50), lgb.log_evaluation(0)]
        )
        
        val_pred = model.predict(X_val)
        cv_score = mean_absolute_error(y_val, val_pred)
        cv_scores.append(cv_score)
        models.append(model)
        
        print(f"----Fold {fold+1} Complete / MAE = {cv_score:.4f}", flush=True)
    
    cv_mean = np.mean(cv_scores)
    print(f"===CV: {cv_mean:.4f} ± {np.std(cv_scores):.3f}===")
    return models, scaler, feature_names, cv_mean

In [15]:
# Define all target properties
targets = ['Tg', 'FFV', 'Tc', 'Density', 'Rg']

# Store trained models and scalers
trained_models = {}
trained_scalers = {}
cv_scores = []

# Train each target - collect results for summary
for target in targets:
    print(f"Training {target}...")
    models, scaler, features, cv_score = train_target_property(X, train_df, target)
    trained_models[target] = models
    trained_scalers[target] = scaler
    cv_scores.append(cv_score)
    print()

# Clean summary with average
print("=" * 40)
print(f"Trained: {len(targets)} targets × 5 CV folds = {len(targets) * 5} models")
print(f"Average CV MAE across all targets: {np.mean(cv_scores):.4f}")

Training Tg...
📊 Training on 511 samples with Tg values
📈 Target range: -148.0297 to 472.2500
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[45]	valid_0's l1: 53.4334
----Fold 1 Complete / MAE = 53.4334
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[120]	valid_0's l1: 57.8898
----Fold 2 Complete / MAE = 57.8898
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[48]	valid_0's l1: 49.6651
----Fold 3 Complete / MAE = 49.6651
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[35]	valid_0's l1: 43.8116
----Fold 4 Complete / MAE = 43.8116
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[41]	valid_0's l1: 45.8876
----Fold 5 Complete / MAE = 45.8876
===CV: 50.1375 ± 5.085===

Training FFV...
📊 Training on 7030 samples with FFV values
📈 Target range: 0.2270 t

In [16]:
def predict_target_property(test_df, target_name, models, scaler):
    
    print(f"PREDICTING: {target_name}")
    
    if models is None or scaler is None:
        print(f"❌ No trained model available for {target_name}, returning zeros")
        return np.zeros(len(test_df))
    
    # Get molecular features - step by step
    descriptor_functions, _ = molecular_descriptors
    X_raw = smiles_to_features(test_df['SMILES'].values, descriptor_functions)
    X = clean_features(X_raw)
    
    # Scale features using same scaler from training
    X_scaled = scaler.transform(X)
    
    # Average predictions from all CV folds
    fold_predictions = []
    for model in models:
        pred = model.predict(X_scaled)
        fold_predictions.append(pred)
    
    predictions = np.mean(fold_predictions, axis=0)
    print(f"📊 Predictions range: {predictions.min():.4f} to {predictions.max():.4f}")
    
    return predictions

In [17]:
print(f"\nMAKING PREDICTIONS...")
all_predictions = {}
for target in targets:
    predictions = predict_target_property(
        test_df, target, trained_models[target], trained_scalers[target]
    )
    all_predictions[target] = predictions

# Create submission
submission = pd.DataFrame({'id': test_df['id']})
for target in targets:
    submission[target] = all_predictions[target]

submission.to_csv('submission.csv', index=False)

print(f"Predicted: {len(test_df)} test samples")
print(f"Saved: submission.csv")

print(f"\n👀 SUBMISSION PREVIEW:")
print(submission.head().to_string(index=False, float_format='%.4f'))


MAKING PREDICTIONS...
PREDICTING: Tg
Processing 3 SMILES... 3/3 ✅
🧹 Cleaned 37 missing values (5.4%)
📊 Predictions range: 80.4436 to 150.5664
PREDICTING: FFV
Processing 3 SMILES... 3/3 ✅
🧹 Cleaned 37 missing values (5.4%)
📊 Predictions range: 0.3498 to 0.3776
PREDICTING: Tc
Processing 3 SMILES... 3/3 ✅
🧹 Cleaned 37 missing values (5.4%)
📊 Predictions range: 0.1822 to 0.2656
PREDICTING: Density
Processing 3 SMILES... 3/3 ✅
🧹 Cleaned 37 missing values (5.4%)
📊 Predictions range: 1.0847 to 1.1396
PREDICTING: Rg
Processing 3 SMILES... 3/3 ✅
🧹 Cleaned 37 missing values (5.4%)
📊 Predictions range: 18.8712 to 19.8186
Predicted: 3 test samples
Saved: submission.csv

👀 SUBMISSION PREVIEW:
        id       Tg    FFV     Tc  Density      Rg
1109053969 138.9369 0.3728 0.1822   1.1396 18.8712
1422188626 150.5664 0.3776 0.2438   1.0847 19.8186
2032016830  80.4436 0.3498 0.2656   1.0986 19.6219
